In [ ]:
import numpy as np

import math
from scipy.sparse import csr_matrix

### PROBLEM GENERATOR

In [3]:
def generate_qkp_instance(n=20, c=30, seed=42):
    """
    Generate a Quadratic Knapsack Problem instance
    
    Maximize: Σ_{i≤j} P_{ij} x_i x_j
    Subject to: Σ w_i x_i ≤ c
    """
    np.random.seed(seed)
    
    # Generate random weights w_i ∈ {1, 2, ..., 10}
    weights = np.random.randint(1, 11, size=n)
    
    # Generate profit matrix P (upper triangular)
    P = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            if i == j:
                P[i, j] = np.random.randint(0, 11)  # Diagonal profits
            else:
                P[i, j] = np.random.randint(0, 11)  # Off-diagonal profits
    
    print(f"   QUADRATIC KNAPSACK PROBLEM (n={n}, c={c})")
    print(f"   Weights: {weights}")
    print(f"   Capacity: {c}")
    print(f"   Profit matrix P shape: {P.shape}")
    print(f"   Example profits: P[0,0]={P[0,0]}, P[0,1]={P[0,1]}, P[1,1]={P[1,1]}")
    
    return weights, P

### BINARY

In [4]:
def create_binary_qubo(weights, P, c):
    """
    Binary encoding: slack variable s = Σ 2^k y_k
    Variables: [x_0, x_1, ..., x_19, y_0, y_1, y_2, y_3, y_4]
    Total: 20 + 5 = 25 variables
    """
    n = len(weights)
    slack_bits = math.ceil(math.log2(c + 1))  # ⌈log₂(31)⌉ = 5
    total_vars = n + slack_bits
    
    print(f"\n🟦 BINARY ENCODING QUBO")
    print(f"   Item variables: {n}")
    print(f"   Slack bits: {slack_bits} (for values 0-{c})")
    print(f"   Total variables: {total_vars}")
    
    # Initialize QUBO matrix
    Q = np.zeros((total_vars, total_vars))
    
    # === OBJECTIVE FUNCTION (MAXIMIZE → MINIMIZE) ===
    # Add -P_{ij} terms (negative because we want to maximize)
    for i in range(n):
        for j in range(i, n):
            Q[i, j] -= P[i, j]  # x_i * x_j term
    
    # === CONSTRAINT: (Σ w_i x_i + s - c)² ===
    A = 1000  # Constraint penalty weight
    
    # Coefficient vector for constraint: [w_0, w_1, ..., w_19, 1, 2, 4, 8, 16]
    constraint_coeffs = np.zeros(total_vars)
    constraint_coeffs[:n] = weights  # Item weights
    constraint_coeffs[n:] = [2**k for k in range(slack_bits)]  # Binary coefficients
    
    # Add constraint: (Σ coeff_i * z_i - c)²
    for i in range(total_vars):
        for j in range(i, total_vars):
            if i == j:
                # Diagonal: coeff_i² * z_i² = coeff_i² * z_i (since z_i² = z_i for binary)
                Q[i, j] += A * constraint_coeffs[i]**2
                # Linear term: -2*c*coeff_i * z_i
                Q[i, j] += A * (-2 * c * constraint_coeffs[i])
            else:
                # Off-diagonal: 2 * coeff_i * coeff_j * z_i * z_j
                Q[i, j] += A * 2 * constraint_coeffs[i] * constraint_coeffs[j]
    
    # Constant term c² is omitted (doesn't affect optimization)
    
    print(f"   QUBO matrix shape: {Q.shape}")
    print(f"   Non-zero elements: {np.count_nonzero(Q)}")
    print(f"   Constraint penalty weight A: {A}")
    
    return Q, total_vars, {'slack_bits': slack_bits, 'coefficients': constraint_coeffs}

### UNARY

In [5]:
def create_unary_qubo(weights, P, c):
    """
    Unary encoding: slack variable s = Σ y_k
    Variables: [x_0, x_1, ..., x_19, y_0, y_1, ..., y_29]
    Total: 20 + 30 = 50 variables
    """
    n = len(weights)
    slack_bits = c  # 30 bits for values 0-30
    total_vars = n + slack_bits
    
    print(f"\n🔵 UNARY ENCODING QUBO")
    print(f"   Item variables: {n}")
    print(f"   Slack bits: {slack_bits} (for values 0-{c})")
    print(f"   Total variables: {total_vars}")
    
    # Initialize QUBO matrix
    Q = np.zeros((total_vars, total_vars))
    
    # === OBJECTIVE FUNCTION ===
    for i in range(n):
        for j in range(i, n):
            Q[i, j] -= P[i, j]
    
    # === CONSTRAINT ===
    A = 1000
    
    # Coefficient vector: [w_0, w_1, ..., w_19, 1, 1, ..., 1]
    constraint_coeffs = np.zeros(total_vars)
    constraint_coeffs[:n] = weights
    constraint_coeffs[n:] = 1  # All slack coefficients are 1
    
    # Add constraint terms
    for i in range(total_vars):
        for j in range(i, total_vars):
            if i == j:
                Q[i, j] += A * constraint_coeffs[i]**2
                Q[i, j] += A * (-2 * c * constraint_coeffs[i])
            else:
                Q[i, j] += A * 2 * constraint_coeffs[i] * constraint_coeffs[j]
    
    print(f"   QUBO matrix shape: {Q.shape}")
    print(f"   Non-zero elements: {np.count_nonzero(Q)}")
    
    return Q, total_vars, {'slack_bits': slack_bits, 'coefficients': constraint_coeffs}

### ONEHOT

In [6]:
def create_onehot_qubo(weights, P, c):
    """
    One-hot encoding: slack variable s = Σ k*y_k with Σ y_k = 1
    Variables: [x_0, x_1, ..., x_19, y_0, y_1, ..., y_30]
    Total: 20 + 31 = 51 variables
    """
    n = len(weights)
    slack_bits = c + 1  # 31 bits for values 0, 1, ..., 30
    total_vars = n + slack_bits
    
    print(f"\n🔴 ONE-HOT ENCODING QUBO")
    print(f"   Item variables: {n}")
    print(f"   Slack bits: {slack_bits} (for values 0-{c})")
    print(f"   Total variables: {total_vars}")
    
    # Initialize QUBO matrix
    Q = np.zeros((total_vars, total_vars))
    
    # === OBJECTIVE FUNCTION ===
    for i in range(n):
        for j in range(i, n):
            Q[i, j] -= P[i, j]
    
    # === MAIN CONSTRAINT: (Σ w_i x_i + Σ k*y_k - c)² ===
    A = 1000
    
    # Coefficient vector: [w_0, w_1, ..., w_19, 0, 1, 2, ..., 30]
    constraint_coeffs = np.zeros(total_vars)
    constraint_coeffs[:n] = weights
    constraint_coeffs[n:] = np.arange(slack_bits)  # [0, 1, 2, ..., 30]
    
    # Add main constraint terms
    for i in range(total_vars):
        for j in range(i, total_vars):
            if i == j:
                Q[i, j] += A * constraint_coeffs[i]**2
                Q[i, j] += A * (-2 * c * constraint_coeffs[i])
            else:
                Q[i, j] += A * 2 * constraint_coeffs[i] * constraint_coeffs[j]
    
    # === ONE-HOT CONSTRAINT: (Σ y_k - 1)² = 0 ===
    B = 1000  # One-hot penalty weight
    
    slack_start = n  # Index where slack variables start
    
    # Add one-hot constraint: (Σ y_k - 1)²
    for i in range(slack_start, total_vars):
        for j in range(i, total_vars):
            if i == j:
                # Diagonal: y_k² - 2*y_k = y_k - 2*y_k = -y_k
                Q[i, j] += B * (-1)
            else:
                # Off-diagonal: 2*y_i*y_j
                Q[i, j] += B * 2
    
    # Constant term 1 is omitted
    
    print(f"   QUBO matrix shape: {Q.shape}")
    print(f"   Non-zero elements: {np.count_nonzero(Q)}")
    print(f"   Main constraint penalty A: {A}")
    print(f"   One-hot constraint penalty B: {B}")
    
    return Q, total_vars, {'slack_bits': slack_bits, 'coefficients': constraint_coeffs}

### BOUNDED (TODO)

In [ ]:
def create_bounded_qubo(weights, P, c, mu=8):
    """
    Bounded encoding with coefficient upper bound μ
    This is more complex - simplified version here
    """
    n = len(weights)
    
    # For bounded encoding, we need to compute the actual encoding
    # This is complex, so I'll show the structure
    print(f"\n🟩 BOUNDED ENCODING QUBO (μ={mu})")
    print(f"   This requires the bounded coefficient algorithm")
    print(f"   Estimated variables: ~35-40 (depends on μ and algorithm)")
    
    # Simplified placeholder - in practice you'd use your bounded encoding functions
    return None, None, None

### DECODERS

In [7]:
def decode_binary_solution(solution_vector, n, slack_bits):
    """Decode solution from binary encoding"""
    x_items = solution_vector[:n]
    y_slack = solution_vector[n:n+slack_bits]
    
    # Decode slack value
    slack_value = sum(bit * (2**k) for k, bit in enumerate(y_slack))
    
    return x_items, slack_value

def decode_unary_solution(solution_vector, n, c):
    """Decode solution from unary encoding"""
    x_items = solution_vector[:n]
    y_slack = solution_vector[n:n+c]
    
    # Decode slack value (sum of bits)
    slack_value = sum(y_slack)
    
    return x_items, slack_value

def decode_onehot_solution(solution_vector, n, c):
    """Decode solution from one-hot encoding"""
    x_items = solution_vector[:n]
    y_slack = solution_vector[n:n+c+1]
    
    # Decode slack value (position of the 1)
    slack_value = sum(k * bit for k, bit in enumerate(y_slack))
    
    # Check if one-hot constraint is satisfied
    one_hot_valid = sum(y_slack) == 1
    
    return x_items, slack_value, one_hot_valid

In [8]:
def run_complete_qubo_example():
    """
    Generate and display complete QUBO formulations for all encodings
    """
    print("=" * 80)
    print("🧮 COMPLETE QUBO FORMULATION EXAMPLE")
    print("=" * 80)
    
    # Generate problem instance
    weights, P = generate_qkp_instance(n=20, c=30)
    
    # Create QUBO matrices for each encoding
    qubo_binary, vars_binary, info_binary = create_binary_qubo(weights, P, 30)
    qubo_unary, vars_unary, info_unary = create_unary_qubo(weights, P, 30) 
    qubo_onehot, vars_onehot, info_onehot = create_onehot_qubo(weights, P, 30)
    
    # Summary
    print(f"\n📊 SUMMARY OF QUBO FORMULATIONS")
    print("=" * 50)
    print(f"{'Encoding':<12} {'Variables':<10} {'Matrix Size':<12} {'Non-zeros':<10}")
    print("-" * 50)
    print(f"{'Binary':<12} {vars_binary:<10} {qubo_binary.shape[0]}×{qubo_binary.shape[1]:<8} {np.count_nonzero(qubo_binary):<10}")
    print(f"{'Unary':<12} {vars_unary:<10} {qubo_unary.shape[0]}×{qubo_unary.shape[1]:<8} {np.count_nonzero(qubo_unary):<10}")
    print(f"{'One-hot':<12} {vars_onehot:<10} {qubo_onehot.shape[0]}×{qubo_onehot.shape[1]:<8} {np.count_nonzero(qubo_onehot):<10}")
    
    # Show example QUBO structure (first 10x10 block)
    print(f"\n🔍 EXAMPLE: Binary Encoding QUBO Matrix (first 10×10):")
    print(qubo_binary[:10, :10])
    
    return {
        'binary': (qubo_binary, vars_binary, info_binary),
        'unary': (qubo_unary, vars_unary, info_unary),
        'onehot': (qubo_onehot, vars_onehot, info_onehot),
        'weights': weights,
        'P': P
    }

### VALIDATION

In [12]:
def validate_solution(x_items, slack_value, weights, c):
    """Validate if a solution satisfies the knapsack constraint"""
    total_weight = sum(w * x for w, x in zip(weights, x_items))
    constraint_satisfied = (total_weight + slack_value == c)
    
    print(f"   Items selected: {sum(x_items)}")
    print(f"   Total weight: {total_weight}")
    print(f"   Slack value: {slack_value}")
    print(f"   Constraint check: {total_weight} + {slack_value} = {total_weight + slack_value} {'==' if constraint_satisfied else '!='} {c}")
    print(f"   Valid solution: {'✅' if constraint_satisfied else '❌'}")
    
    return constraint_satisfied

def calculate_objective(x_items, P):
    """Calculate the objective value for a solution"""
    n = len(x_items)
    objective = 0
    
    for i in range(n):
        for j in range(i, n):
            objective += P[i, j] * x_items[i] * x_items[j]
    
    return objective

In [13]:
if __name__ == "__main__":
    results = run_complete_qubo_example()
    
    print(f"\n✅ QUBO matrices generated successfully!")
    print(f"💾 Results stored in 'results' dictionary")
    print(f"🚀 Ready for simulated annealing experiments!")

🧮 COMPLETE QUBO FORMULATION EXAMPLE
   QUADRATIC KNAPSACK PROBLEM (n=20, c=30)
   Weights: [ 7  4  8  5  7 10  3  7  8  5  4  8  8  3  6  5  2  8  6  2]
   Capacity: 30
   Profit matrix P shape: (20, 20)
   Example profits: P[0,0]=4.0, P[0,1]=0.0, P[1,1]=1.0

🟦 BINARY ENCODING QUBO
   Item variables: 20
   Slack bits: 5 (for values 0-30)
   Total variables: 25
   QUBO matrix shape: (25, 25)
   Non-zero elements: 325
   Constraint penalty weight A: 1000

🔵 UNARY ENCODING QUBO
   Item variables: 20
   Slack bits: 30 (for values 0-30)
   Total variables: 50
   QUBO matrix shape: (50, 50)
   Non-zero elements: 1275

🔴 ONE-HOT ENCODING QUBO
   Item variables: 20
   Slack bits: 31 (for values 0-30)
   Total variables: 51
   QUBO matrix shape: (51, 51)
   Non-zero elements: 1306
   Main constraint penalty A: 1000
   One-hot constraint penalty B: 1000

📊 SUMMARY OF QUBO FORMULATIONS
Encoding     Variables  Matrix Size  Non-zeros 
--------------------------------------------------
Binary       

In [20]:
results["binary"][0].shape, results["onehot"][0].shape, results["unary"][0].shape

((25, 25), (51, 51), (50, 50))

In [21]:
import dimod
from neal import SimulatedAnnealingSampler

In [24]:
binaryBqm = dimod.BQM.from_qubo(results["binary"][0])
unaryBqm = dimod.BQM.from_qubo(results["unary"][0])
onehotBqm = dimod.BQM.from_qubo(results["onehot"][0])

bqms = [binaryBqm, unaryBqm, onehotBqm]

In [27]:
energies = {
    "binary" : [],
    "unary" : [],
    "onehot" : []
}

for _ in range(25):
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(binaryBqm, num_reads= 100)
    energies["binary"].append(sampleset.first.energy)

    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(unaryBqm, num_reads=100)
    energies["unary"].append(sampleset.first.energy)

    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(onehotBqm, num_reads = 100)
    energies["onehot"].append(sampleset.first.energy)

In [28]:
binaryEnergies = np.array(energies["binary"])
unaryEnergies = np.array(energies["unary"])
onehotEnergies = np.array(energies["onehot"])

In [29]:
np.mean(binaryEnergies), np.std(binaryEnergies)

(np.float64(-900157.6), np.float64(10.947145746723205))

In [30]:
np.mean(unaryEnergies), np.std(unaryEnergies)

(np.float64(-900105.32), np.float64(10.42197677986283))

In [31]:
np.mean(onehotEnergies), np.std(onehotEnergies)

(np.float64(-901159.6), np.float64(18.706148721743876))

In [ ]:
import numpy as np
from typing import List

def compare_encodings_simple():
    """
    Simple example comparing the three encodings for a small QKP instance.
    """
    
    # Small QKP instance
    weights = [2, 3]  # 2 items
    profits = np.array([[5, 2], [2, 4]])  # 2x2 profit matrix
    capacity = 4
    
    print("=== SIMPLE QKP INSTANCE ===")
    print(f"Items: 2")
    print(f"Weights: {weights}")
    print(f"Profits: {profits.tolist()}")
    print(f"Capacity: {capacity}")
    print()
    
    formulator = QKPQUBOFormulator(weights, profits, capacity)
    
    # Compare encodings
    encodings = [
        ("One-hot", "one_hot"),
        ("Binary", "binary"), 
        ("Unary", "unary")
    ]
    
    A = 5.0  # Penalty parameter
    
    for name, encoding_type in encodings:
        Q, var_info = formulator.formulate_qubo(encoding_type, A)
        
        print(f"=== {name.upper()} ENCODING ===")
        print(f"Bit depth D: {var_info['bit_depth']}")
        print(f"Total variables: {var_info['total_vars']}")
        print(f"Variables: x₀, x₁, y₀, y₁, ..." + ("" if var_info['bit_depth'] <= 2 else f", y_{var_info['bit_depth']-1}"))
        print(f"QUBO matrix Q:")
        
        # Print matrix with variable labels
        n = var_info['total_vars']
        var_labels = [f"x{i}" for i in range(var_info['n_items'])] + \
                    [f"y{i}" for i in range(var_info['bit_depth'])]
        
        print("    " + "".join(f"{label:>8}" for label in var_labels))
        for i in range(n):
            row_str = f"{var_labels[i]:<3} "
            for j in range(n):
                row_str += f"{Q[i,j]:>8.1f}"
            print(row_str)
        print()


def demonstrate_constraint_representations():
    """
    Show how each encoding represents the constraint for capacity = 4.
    """
    print("=== CONSTRAINT REPRESENTATIONS (capacity = 4) ===")
    print()
    
    print("One-hot encoding (D = 5, f(d) = d):")
    print("  Integer I = y₀*0 + y₁*1 + y₂*2 + y₃*3 + y₄*4")
    print("  Constraint: exactly one yₑ = 1")
    print("  Examples: I=0 → [1,0,0,0,0], I=2 → [0,0,1,0,0], I=4 → [0,0,0,0,1]")
    print()
    
    print("Binary encoding (D = 2, f(d) = 2ᵈ):")
    print("  Integer I = y₀*2⁰ + y₁*2¹ - offset")
    print("  where offset = 2² - 1 - 4 = -1")
    print("  So I = y₀ + 2*y₁ + 1")
    print("  Examples: I=0 → [1,1], I=2 → [1,0], I=4 → [1,0] (but constrained)")
    print()
    
    print("Unary encoding (D = 4, f(d) = 1):")
    print("  Integer I = y₀ + y₁ + y₂ + y₃")
    print("  Examples: I=0 → [0,0,0,0], I=2 → [1,1,0,0], I=4 → [1,1,1,1]")
    print("  Note: Multiple representations possible (e.g., I=2 could be [0,1,1,0])")
    print()


if __name__ == "__main__":
    # Import the main class
    from __main__ import QKPQUBOFormulator
    
    compare_encodings_simple()
    demonstrate_constraint_representations()

In [ ]:
import numpy as np
from typing import Dict, Tuple, List
import math




def example_usage():
    """Example usage of the QUBO formulator."""
    
    # Example QKP instance
    weights = [2, 3, 4, 5]
    profits = np.array([
        [10, 5, 3, 2],
        [5, 8, 4, 1],
        [3, 4, 12, 6],
        [2, 1, 6, 9]
    ])
    capacity = 7
    
    formulator = QKPQUBOFormulator(weights, profits, capacity)
    
    print("QKP Instance:")
    print(f"Weights: {weights}")
    print(f"Capacity: {capacity}")
    print(f"Profits matrix:\n{profits}")
    print()
    
    # Generate QUBO for each encoding
    encodings = ["one_hot", "binary", "unary"]
    A = 10.0  # Constraint penalty parameter
    
    for encoding in encodings:
        Q, var_info = formulator.formulate_qubo(encoding, A)
        
        print(f"{encoding.upper()} ENCODING:")
        print(f"  Bit depth D: {var_info['bit_depth']}")
        print(f"  Total variables: {var_info['total_vars']}")
        print(f"  QUBO matrix shape: {Q.shape}")
        print(f"  Variable indices - Items: {var_info['x_indices']}, Auxiliary: {var_info['y_indices']}")
        print(f"  QUBO matrix:\n{Q}")
        print()


if __name__ == "__main__":
    example_usage()